In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import pandas as pd
from DateProvider import DateProvider
from datetime import datetime
from ColumnsAdder import ColumnsAdder
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries
from IntensivstationenHtmlFileUpdater import saveLastUpdatedIntensivstationen, saveKreisOptions
from KreiseReader import readKreise
from KreisOptionsProvider import getKreisOptionsAndAlleLandkreise
from IntensiveCareBedsPersister import getAndPersistIntensiveCareBeds4AlleKreise
from Chart import createMedianOfFreeBedsByKreisChart


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [3]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

25.04.2024, 11:28:10 Uhr


In [4]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = True # dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

          lastUpdated: 2023-02-03 11:49:00


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5608a1a2fce3 <unknown>
#1 0x5608a171e4a7 <unknown>
#2 0x5608a1752a26 <unknown>
#3 0x5608a174e980 <unknown>
#4 0x5608a17977b0 <unknown>
#5 0x5608a178b1b3 <unknown>
#6 0x5608a175c24a <unknown>
#7 0x5608a175cc1e <unknown>
#8 0x5608a19f400b <unknown>
#9 0x5608a19f7f5b <unknown>
#10 0x5608a19e0121 <unknown>
#11 0x5608a19f8ac2 <unknown>
#12 0x5608a19c4fdf <unknown>
#13 0x5608a1a1ee38 <unknown>
#14 0x5608a1a1f010 <unknown>
#15 0x5608a1a2ee34 <unknown>
#16 0x7f728f694ac3 <unknown>


In [5]:
saveLastUpdatedIntensivstationen(
    dateProvider.getLastUpdatedDataSource(),
    toHtmlFile = "../../docs/intensivstationen.html")

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55f287e52ce3 <unknown>
#1 0x55f287b414a7 <unknown>
#2 0x55f287b75a26 <unknown>
#3 0x55f287b71980 <unknown>
#4 0x55f287bba7b0 <unknown>
#5 0x55f287bae1b3 <unknown>
#6 0x55f287b7f24a <unknown>
#7 0x55f287b7fc1e <unknown>
#8 0x55f287e1700b <unknown>
#9 0x55f287e1af5b <unknown>
#10 0x55f287e03121 <unknown>
#11 0x55f287e1bac2 <unknown>
#12 0x55f287de7fdf <unknown>
#13 0x55f287e41e38 <unknown>
#14 0x55f287e42010 <unknown>
#15 0x55f287e51e34 <unknown>
#16 0x7ff7f7894ac3 <unknown>


In [6]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

NameError: name 'needsUpdate' is not defined

In [7]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)
saveKreisOptions(
    kreisOptions = getKreisOptionsAndAlleLandkreise(kreisValues),
    toHtmlFile = "../../docs/intensivstationen.html")

NameError: name 'timeSeries' is not defined

In [8]:
getAndPersistIntensiveCareBeds4AlleKreise(
    timeSeries,
    intensivstationenDataDir = '../../docs/data/intensivstationen',
    kreisValues = kreisValues)


NameError: name 'timeSeries' is not defined

In [9]:
medianOfFreeBedsByKreisTable = MedianOfFreeBedsByKreisTableFactory(timeSeries).createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json(
    '../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json',
    orient = "records")

NameError: name 'timeSeries' is not defined

In [10]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [11]:
import os

createMedianOfFreeBedsByKreisChart(
    timeSeries,
    chartTitle = getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()),
    accessToken = os.environ['DATAWRAPPER_API_TOKEN'])

NameError: name 'timeSeries' is not defined